In [3]:
import sys 
import os 
import pickle
import random
import numpy as np 
from astropy.io import fits
from astropy.coordinates import SkyCoord  
import astropy.units as u
import matplotlib.pylab as plt 
import pandas as pd 

plt.rc('text', usetex=True) # - Imperial cluster cannot find Latex
plt.rc('font',**{'family':'sans-serif','serif':['Palatino']})
figSize  = (12, 8)
fontSize = 20

In [ ]:
fits_image = fits.open('./data/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits')
data = fits_image[1].data
fits_image.close()

# Notes
Important columns identified in the catalogue.

## Positions on the sky
- ALPHA_J2000 (deg)
- DELTA_J2000 (deg)

## Ellipticities and Weights
- e1
- e2
- weight

## Flux
- FLUX_GAAP_u (count)
- FLUX_GAAP_g (count)
- FLUX_GAAP_r (count)
- FLUX_GAAP_i (count)
- FLUX_GAAP_Z (count)
- FLUX_GAAP_Y (count)
- FLUX_GAAP_J (count)
- FLUX_GAAP_H (count)
- FLUX_GAAP_Ks (count)

## Flux Errors
- FLUXERR_GAAP_u (count)
- FLUXERR_GAAP_g (count)
- FLUXERR_GAAP_r (count)
- FLUXERR_GAAP_i (count)
- FLUXERR_GAAP_Z (count)
- FLUXERR_GAAP_Y (count)
- FLUXERR_GAAP_J (count)
- FLUXERR_GAAP_H (count)
- FLUXERR_GAAP_Ks (count)

## Magnitude
- MAG_GAAP_u (mag)
- MAG_GAAP_g (mag)
- MAG_GAAP_r (mag)
- MAG_GAAP_i (mag)
- MAG_GAAP_Z (mag)
- MAG_GAAP_Y (mag)
- MAG_GAAP_J (mag)
- MAG_GAAP_H (mag)
- MAG_GAAP_Ks (mag)

## Magnitude Error
- MAGERR_GAAP_u (mag)
- MAGERR_GAAP_g (mag)
- MAGERR_GAAP_r (mag)
- MAGERR_GAAP_i (mag)
- MAGERR_GAAP_Z (mag)
- MAGERR_GAAP_Y (mag)
- MAGERR_GAAP_J (mag)
- MAGERR_GAAP_H (mag)
- MAGERR_GAAP_Ks (mag)

## Magnitude Limit
- MAG_LIM_u (mag)
- MAG_LIM_g (mag)
- MAG_LIM_r (mag)
- MAG_LIM_i (mag)
- MAG_LIM_Z (mag)
- MAG_LIM_Y (mag)
- MAG_LIM_J (mag)
- MAG_LIM_H (mag)
- MAG_LIM_Ks (mag)

## Flag (all of them 0)
- FLAG_GAAP_u
- FLAG_GAAP_g
- FLAG_GAAP_r
- FLAG_GAAP_i
- FLAG_GAAP_Z
- FLAG_GAAP_Y
- FLAG_GAAP_J
- FLAG_GAAP_H
- FLAG_GAAP_Ks

## BPZ
- M_0 (reference magnitude for BPZ prior)
- Z_B (9 band BPZ redshift estimate - peak of posterior)
- Z_ML (9 band BPZ maximum likelihood redshift)
- Z_B_MIN (lower bound of the 68% confidence interval of Z_B)
- Z_B_MAX (upper bound of the 68% confidence interval of Z_B)

# Plot Objects on the Sky

We have chosen the first 500 000 objects in the catalogue. Takes a long time to plot all the samples. 

In [ ]:
xarr, yarr = data['ALPHA_J2000'], data['DELTA_J2000']
nobjects = 1000000
idx = random.sample(range(len(xarr)), nobjects)
eq = SkyCoord(xarr[idx], yarr[idx], unit=u.deg)
gal = eq.galactic
colors = np.random.random((nobjects, 3))

In [ ]:
plt.figure(figsize = (12, 8))
plt.subplot(111, projection='aitoff')
plt.grid(True)
plt.scatter(gal.l.wrap_at('180d').radian, gal.b.radian, s=10, c=colors, alpha=0.6, edgecolors=colors, rasterized=True)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
# plt.savefig('plots/skymap.pdf', bbox_inches = 'tight')
plt.show()

# Choosing Specific Columns

In [ ]:
band = ['u', 'g', 'r', 'i', 'Z', 'Y', 'J', 'H', 'Ks']
flux_names = [f'FLUX_GAAP_{b}' for b in band]
flux_names_err = [f'FLUXERR_GAAP_{b}' for b in band]
mag_names = [f'MAG_GAAP_{b}' for b in band]
mag_names_err =  [f'MAGERR_GAAP_{b}' for b in band]
bpz_names = ['Z_B', 'M_0', 'Z_ML', 'Z_B_MIN', 'Z_B_MAX']
meta_names = ['ALPHA_J2000', 'DELTA_J2000', 'e1', 'e2', 'weight']

nbpz = len(bpz_names)
nmeta = len(meta_names)
nband = len(band)

In [ ]:
fluxes = np.asarray([data[flux_names[i]] for i in range(nband)]).T
fluxes_err = np.asarray([data[flux_names_err[i]] for i in range(nband)]).T
mag = np.asarray([data[mag_names[i]] for i in range(nband)]).T
mag_err = np.asarray([data[mag_names_err[i]] for i in range(nband)]).T
bpz = np.asarray([data[mag_names_err[i]] for i in range(nbpz)]).T
meta = np.asarray([data[meta_names[i]] for i in range(nmeta)]).T

In [ ]:
df_flux = pd.DataFrame(fluxes, columns = flux_names, dtype=np.float32)
df_flux_err = pd.DataFrame(fluxes_err, columns = flux_names_err, dtype=np.float32)
df_mag = pd.DataFrame(mag, columns = mag_names, dtype=np.float16)
df_mag_err = pd.DataFrame(mag_err, columns = mag_names_err, dtype=np.float16)
df_bpz = pd.DataFrame(bpz, columns = bpz_names, dtype=np.float16)
df_meta = pd.DataFrame(meta, columns = meta_names, dtype=np.float16)

## Idea about the size of the dataframes

In [ ]:
def get_size_mb(dataframe: pd.DataFrame) -> float:
    print(f'Size of dataframe in MB is {sys.getsizeof(dataframe)/1024**2:.2f}')    

In [ ]:
get_size_mb(df_flux)

In [ ]:
get_size_mb(df_flux_err)

In [ ]:
get_size_mb(df_mag)

In [ ]:
get_size_mb(df_mag_err)

In [ ]:
get_size_mb(df_bpz)

In [ ]:
get_size_mb(df_meta)

## Example of the tabular format

In [ ]:
df_flux.head()

In [ ]:
df_flux_err.head()

In [ ]:
df_mag.head()

In [ ]:
df_mag_err.head()

In [ ]:
df_bpz.head()

In [ ]:
df_meta.head()

## Save the files

In [ ]:
def pickle_save(file: list, folder_name: str, file_name: str) -> None:
    """Stores a list in a folder.
    Args:
        list_to_store (list): The list to store.
        folder_name (str): The name of the folder.
        file_name (str): The name of the file.
    """

    # create the folder if it does not exist
    os.makedirs(folder_name, exist_ok=True)

    # use compressed format to store data
    with open(folder_name + "/" + file_name + ".pkl", "wb") as f:
        pickle.dump(file, f)


def pickle_load(folder_name: str, file_name: str) -> pd.DataFrame:
    """Reads a list from a folder.
    Args:
        folder_name (str): The name of the folder.
        file_name (str): The name of the file.
    Returns:
        pd.DataFrame: a pandas dataframe
    """

    with open(folder_name + "/" + file_name + ".pkl", "rb") as f:
         file = pickle.load(f)

    return file

In [ ]:
# pickle_save(df_flux, 'data/processed', 'flux')
# pickle_save(df_flux_err, 'data/processed', 'flux_err')
# pickle_save(df_mag, 'data/processed', 'mag')
# pickle_save(df_mag_err, 'data/processed', 'mag_err')
# pickle_save(df_meta, 'data/processed', 'meta')
# pickle_save(df_bpz, 'data/processed', 'bpz')

In [1]:
from src.processing import cleaning 

In [ ]:
fits_image = fits.open('./data/KiDS_DR4.1_ugriZYJHKs_SOM_gold_WL_cat.fits')

dictionary = cleaning(fits_image, save = False)

fits_image.close()